# Week 1
## 1.2 Transforming Distance Matrices
In this chapter, we define the length of a path in a tree as the sum of the lengths of its edges (rather than the number of edges on the path). As a result, the evolutionary distance between two present-day species corresponding to leaves i and j in a tree T is equal to the length of the unique path connecting i and j, denoted di,j(T).

Distances Between Leaves Problem: Compute the distances between leaves in a weighted tree.

Input:  An integer n followed by the adjacency list of a weighted tree with n leaves.
Output: An n x n matrix (di,j), where di,j is the length of the path between leaves i and j.
Code Challenge: Solve the Distances Between Leaves Problem. The tree is given as an adjacency list of a graph whose leaves are integers between 0 and n - 1; the notation a->b:c means that node a is connected to node b by an edge of weight c. The matrix you return should be space-separated.

Sample Input:

4
0->4:11
1->4:2
2->5:6
3->5:7
4->0:11
4->1:2
4->5:4
5->4:4
5->3:7
5->2:6

Sample Output:

0	13	21	22
13	0	12	13
21	12	0	13
22	13	13	0

In [5]:
from collections import defaultdict, deque

def parse_input(n, edges):
    graph = defaultdict(list)
    for edge in edges:
        a, rest = edge.split("->")
        b, weight = rest.split(":")
        a, b, weight = int(a), int(b), int(weight)
        graph[a].append((b, weight))
    print(f"Parsed graph: {graph}")
    return graph

def bfs(graph, start):
    distances = {}
    queue = deque([(start, 0)])
    visited = set()
    while queue:
        node, dist = queue.popleft()
        if node in visited:
            continue
        visited.add(node)
        distances[node] = dist
        for neighbor, weight in graph[node]:
            if neighbor not in visited:
                queue.append((neighbor, dist + weight))
    print(f"Distances from node {start}: {distances}")
    return distances

def distances_between_leaves(n, edges):
    graph = parse_input(n, edges)
    result = [[0]*n for _ in range(n)]
    
    for i in range(n):
        dist = bfs(graph, i)
        for j in range(n):
            result[i][j] = dist[j]
    print(f"Resulting distance matrix: {result}")
    return result

# Sample input
n = 4
edges = [
    "0->4:11",
    "1->4:2",
    "2->5:6",
    "3->5:7",
    "4->0:11",
    "4->1:2",
    "4->5:4",
    "5->4:4",
    "5->3:7",
    "5->2:6"
]

# Run the function
matrix = distances_between_leaves(n, edges)

# Print the output
for row in matrix:
    print('\t'.join(map(str, row)))

Parsed graph: defaultdict(<class 'list'>, {0: [(4, 11)], 1: [(4, 2)], 2: [(5, 6)], 3: [(5, 7)], 4: [(0, 11), (1, 2), (5, 4)], 5: [(4, 4), (3, 7), (2, 6)]})
Distances from node 0: {0: 0, 4: 11, 1: 13, 5: 15, 3: 22, 2: 21}
Distances from node 1: {1: 0, 4: 2, 0: 13, 5: 6, 3: 13, 2: 12}
Distances from node 2: {2: 0, 5: 6, 4: 10, 3: 13, 0: 21, 1: 12}
Distances from node 3: {3: 0, 5: 7, 4: 11, 2: 13, 0: 22, 1: 13}
Resulting distance matrix: [[0, 13, 21, 22], [13, 0, 12, 13], [21, 12, 0, 13], [22, 13, 13, 0]]
0	13	21	22
13	0	12	13
21	12	0	13
22	13	13	0


## 1.3 Toward An Algorithm for Distance-Based Phylogeny Construction
We now have an algorithm for solving the Limb Length Problem. For each j, we can compute LimbLength(j) by finding the minimum value of 
(Di,j+Dj,k−Di,k)/2 over all pairs of leaves i and k (where i ≠ j and k ≠ j).

Code Challenge: Solve the Limb Length Problem.

Input: An integer n, followed by an integer j between 0 and n - 1, followed by a space-separated additive distance matrix D (whose elements are integers).
Output: The limb length of the leaf in Tree(D) corresponding to row j of this distance matrix (use 0-based indexing).

In [ ]:
def limb_length(n, j, D):
    min_limb = float('inf')
    for i in range(n):
        for k in range(n):
            if i != j and k != j:
                limb = (D[i][j] + D[j][k] - D[i][k]) // 2
                min_limb = min(min_limb, limb)
    return min_limb

# Sample Input
n = 27
j = 2
D = [
    [0, 13, 21, 22],
    [13, 0, 12, 13],
    [21, 12, 0, 13],
    [22, 13, 13, 0]
]

# Call the function and print the result
print(limb_length(n, j, D))  # Output: 2


2


In [7]:
def limb_length(n, j, D):
    min_limb_len = float('inf')
    for i in range(n):
        for k in range(n):
            if i != j and k != j and i != k:
                val = (D[i][j] + D[j][k] - D[i][k]) // 2
                if val < min_limb_len:
                    min_limb_len = val
    return min_limb_len

# Input values
n = 27
j = 2
flat_data = [
    0,6829,3222,10993,4119,4856,8910,13439,9473,6564,7952,8479,8667,9736,10653,12396,4407,3310,12997,3301,1176,5232,2297,1410,1570,11740,1890,
    6829,0,9247,4830,3732,3393,2747,7276,3310,2087,1789,2316,2504,3573,4490,6233,4314,9335,6834,4726,7201,2665,8322,6315,7595,5577,5177,
    3222,9247,0,13411,6537,7274,11328,15857,11891,8982,10370,10897,11085,12154,13071,14814,6825,1080,15415,5719,2572,7650,1937,3828,3200,14158,4308,
    10993,4830,13411,0,7896,7557,3715,3466,2290,6251,4287,3714,2666,1579,1972,2423,8478,13499,3024,8890,11365,6829,12486,10479,11759,1767,9341,
    4119,3732,6537,7896,0,1759,5813,10342,6376,3467,4855,5382,5570,6639,7556,9299,1604,6625,9900,2016,4491,2135,5612,3605,4885,8643,2467,
    4856,3393,7274,7557,1759,0,5474,10003,6037,3128,4516,5043,5231,6300,7217,8960,2341,7362,9561,2753,5228,1796,6349,4342,5622,8304,3204,
    8910,2747,11328,3715,5813,5474,0,6161,2195,4168,2204,1631,1389,2458,3375,5118,6395,11416,5719,6807,9282,4746,10403,8396,9676,4462,7258,
    13439,7276,15857,3466,10342,10003,6161,0,4736,8697,6733,6160,5112,4025,4418,2189,10924,15945,876,11336,13811,9275,14932,12925,14205,3331,11787,
    9473,3310,11891,2290,6376,6037,2195,4736,0,4731,2767,2194,1146,1033,1950,3693,6958,11979,4294,7370,9845,5309,10966,8959,10239,3037,7821,
    6564,2087,8982,6251,3467,3128,4168,8697,4731,0,3210,3737,3925,4994,5911,7654,4049,9070,8255,4461,6936,2400,8057,6050,7330,6998,4912,
    7952,1789,10370,4287,4855,4516,2204,6733,2767,3210,0,1773,1961,3030,3947,5690,5437,10458,6291,5849,8324,3788,9445,7438,8718,5034,6300,
    8479,2316,10897,3714,5382,5043,1631,6160,2194,3737,1773,0,1388,2457,3374,5117,5964,10985,5718,6376,8851,4315,9972,7965,9245,4461,6827,
    8667,2504,11085,2666,5570,5231,1389,5112,1146,3925,1961,1388,0,1409,2326,4069,6152,11173,4670,6564,9039,4503,10160,8153,9433,3413,7015,
    9736,3573,12154,1579,6639,6300,2458,4025,1033,4994,3030,2457,1409,0,1239,2982,7221,12242,3583,7633,10108,5572,11229,9222,10502,2326,8084,
    10653,4490,13071,1972,7556,7217,3375,4418,1950,5911,3947,3374,2326,1239,0,3375,8138,13159,3976,8550,11025,6489,12146,10139,11419,2719,9001,
    12396,6233,14814,2423,9299,8960,5118,2189,3693,7654,5690,5117,4069,2982,3375,0,9881,14902,1747,10293,12768,8232,13889,11882,13162,2288,10744,
    4407,4314,6825,8478,1604,2341,6395,10924,6958,4049,5437,5964,6152,7221,8138,9881,0,6913,10482,2304,4779,2717,5900,3893,5173,9225,2755,
    3310,9335,1080,13499,6625,7362,11416,15945,11979,9070,10458,10985,11173,12242,13159,14902,6913,0,15503,5807,2660,7738,2025,3916,3288,14246,4396,
    12997,6834,15415,3024,9900,9561,5719,876,4294,8255,6291,5718,4670,3583,3976,1747,10482,15503,0,10894,13369,8833,14490,12483,13763,2889,11345,
    3301,4726,5719,8890,2016,2753,6807,11336,7370,4461,5849,6376,6564,7633,8550,10293,2304,5807,10894,0,3673,3129,4794,2787,4067,9637,1649,
    1176,7201,2572,11365,4491,5228,9282,13811,9845,6936,8324,8851,9039,10108,11025,12768,4779,2660,13369,3673,0,5604,1647,1782,1154,12112,2262,
    5232,2665,7650,6829,2135,1796,4746,9275,5309,2400,3788,4315,4503,5572,6489,8232,2717,7738,8833,3129,5604,0,6725,4718,5998,7576,3580,
    2297,8322,1937,12486,5612,6349,10403,14932,10966,8057,9445,9972,10160,11229,12146,13889,5900,2025,14490,4794,1647,6725,0,2903,2275,13233,3383,
    1410,6315,3828,10479,3605,4342,8396,12925,8959,6050,7438,7965,8153,9222,10139,11882,3893,3916,12483,2787,1782,4718,2903,0,2176,11226,1376,
    1570,7595,3200,11759,4885,5622,9676,14205,10239,7330,8718,9245,9433,10502,11419,13162,5173,3288,13763,4067,1154,5998,2275,2176,0,12506,2656,
    11740,5577,14158,1767,8643,8304,4462,3331,3037,6998,5034,4461,3413,2326,2719,2288,9225,14246,2889,9637,12112,7576,13233,11226,12506,0,10088,
    1890,5177,4308,9341,2467,3204,7258,11787,7821,4912,6300,6827,7015,8084,9001,10744,2755,4396,11345,1649,2262,3580,3383,1376,2656,10088,0
]

# Convert flat list to 2D distance matrix
D = [flat_data[i * n:(i + 1) * n] for i in range(n)]

# Compute limb length
result = limb_length(n, j, D)
print(result)


496


## 1.4
An algorithm for distance-based phylogeny reconstruction
The preceding discussion results in the following recursive algorithm, called AdditivePhylogeny, for finding the simple tree fitting an n x n additive distance matrix D. We assume that you have already implemented a program Limb(D, j) that computes LimbLength(j) for a leaf j based on the distance matrix D. Rather than selecting an arbitrary leaf j from Tree(D) for trimming, AdditivePhylogeny selects leaf n (corresponding to the last row and column of D).

AdditivePhylogeny(D)
    n ← number of rows in D
    if n = 2
        return the tree consisting of a single edge of length D1,2
    limbLength ← Limb(D, n)
    for j ← 1 to n - 1
        Dj,n ← Dj,n - limbLength
        Dn,j ← Dj,n
    (i, k) ← two leaves such that Di,k = Di,n + Dn,k
    x ← Di,n
    D ← D﻿ with row n and column n removed
    T ← AdditivePhylogeny(D)
    v ← the (potentially new) node in T at distance x from i on the path between i and k
    add leaf n back to T by creating a limb (v, n) of length limbLength
    return T

In [10]:
def limb_length(D, j):
    n = len(D)
    limb_len = float('inf')
    for i in range(n):
        if i == j:
            continue
        for k in range(n):
            if k == j or i == k:
                continue
            limb = (D[i][j] + D[j][k] - D[i][k]) // 2
            if limb < limb_len:
                limb_len = limb
    return limb_len

def find_path(tree, u, v, visited=None, path=None, total_weight=0):
    if visited is None:
        visited = set()
    if path is None:
        path = []

    visited.add(u)
    path.append((u, total_weight))

    if u == v:
        return path

    for neighbor, weight in tree.get(u, []):
        if neighbor not in visited:
            result = find_path(tree, neighbor, v, visited.copy(), path.copy(), weight)
            if result is not None:
                return result
    return None

def insert_node(tree, u, v, x, new_node, distance):
    path = find_path(tree, u, v)
    current_dist = 0

    for i in range(len(path) - 1):
        node1, d1 = path[i]
        node2, d2 = path[i + 1]
        weight = d2

        current_dist += weight
        if current_dist == x:
            return node2
        elif current_dist > x:
            # insert new node
            over_shoot = current_dist - x
            undershoot = weight - over_shoot

            # remove edge
            tree[node1].remove((node2, weight))
            tree[node2].remove((node1, weight))

            # insert new node with two edges
            tree[node1].append((new_node, undershoot))
            tree[new_node] = [(node1, undershoot), (node2, over_shoot)]
            tree[node2].append((new_node, over_shoot))
            return new_node

def additive_phylogeny(D, n, next_node):
    if n == 2:
        tree = {
            0: [(1, D[0][1])],
            1: [(0, D[0][1])]
        }
        return tree

    limb_len = limb_length(D, n - 1)

    for j in range(n - 1):
        D[j][n - 1] -= limb_len
        D[n - 1][j] = D[j][n - 1]

    x, i, k = 0, -1, -1
    for i_ in range(n - 1):
        for k_ in range(n - 1):
            if i_ == k_:
                continue
            if D[i_][k_] == D[i_][n - 1] + D[n - 1][k_]:
                i, k, x = i_, k_, D[i_][n - 1]
                break
        if i != -1:
            break

    D_prime = [row[:-1] for row in D[:-1]]
    tree = additive_phylogeny(D_prime, n - 1, next_node)

    v = insert_node(tree, i, k, x, next_node[0], D[i][k])
    if v == next_node[0]:
        next_node[0] += 1

    tree[v].append((n - 1, limb_len))
    tree[n - 1] = [(v, limb_len)]
    return tree

def format_adjacency_list(tree):
    output = []
    for u in sorted(tree):
        for v, w in tree[u]:
            output.append(f"{u}->{v}:{w}")
    return output

# Example usage
def main():
    input_data = """4
0\t13\t21\t22
13\t0\t12\t13
21\t12\t0\t13
22\t13\t13\t0"""

    lines = input_data.strip().split('\n')
    n = int(lines[0])
    D = [list(map(int, line.split())) for line in lines[1:]]
    tree = additive_phylogeny(D, n, [n])
    output = format_adjacency_list(tree)
    print("\n".join(output))

main()


0->4:11
1->4:2
2->5:6
3->5:7
4->0:11
4->1:2
4->5:4
5->4:4
5->2:6
5->3:7
